# Getting data from HTML

HTML stands for "hypertext markup language." Most of the documents you see when you're browsing the web are written in this format.  Our goal today is to discuss and put into practice basic methods for extracting data from documents written in HTML.

In most browsers, there's a "View Source" option that allows you to see the HTML source code for any page you're looking at. For example, in Chrome, you can CTRL-click anywhere on the page, or go to `View > Developer > View Source`:

<a href="http://static.decontextualize.com/snaps/nytimes-view-source.png"><img src="http://static.decontextualize.com/snaps/nytimes-view-source.png" alt="nytimes-view-source"/></a>

You'll see something that looks like this, a mish-mash of angle brackets and quotes and slashes and text. This is HTML.

<a href="http://static.decontextualize.com/snaps/nytimes-source.png"><img src="http://static.decontextualize.com/snaps/nytimes-source.png" alt="nytimes-source"/></a>

### What HTML looks like

HTML consists of a series of *tags*. Tags have a *name*, a series of key/value pairs called *attributes*, and some textual *content*. Attributes are optional. Here's a simple example, using the HTML `<p>` tag (`p` means "paragraph"):

    <p>Mother said there'd be days like these.</p>
    
This example has just one tag in it: a `<p>` tag. The source code for a tag has two parts, its opening tag (`<p>`) and its closing tag (`</p>`). In between the opening and closing tag, you see the tag's contents (in this case, the text `Mother said there'd be days like these.`).

Here's another example, using the HTML `<div>` tag:

    <div class="header" style="background: blue;">Mammoth Falls</div>
    
In this example, the tag's name is `div`. The tag has two attributes: `class`, with value `header`, and `style`, with value `background: blue;`. The contents of this tag is `Mammoth Falls`.

Tags can contain other tags, in a hierarchical relationship. For example, here's some HTML to make a bulletted list:

    <ul>
      <li>Item one</li>
      <li>Item two</li>
      <li>Item three</li>
    </ul>

The `<ul>` tag (`ul` stands for "unordered list") in this example has three other `<li>` tags inside of it (`li` stands for "list item"). The `<ul>` tag is said to be the "parent" of the `<li>` tags, and the `<li>` tags are the "children" of the `<ul>` tag. All tags grouped under a particular parent tag are called "siblings."

### HTML's shortcomings

HTML documents are intended to add "markup" to text to add information that allows browsers to display the text in different ways---e.g., HTML markup might tell the browser to make the font of the text a particular size, or to position it in a particular place on the screen.

Because the primary purpose of HTML is to change the appearance of text, HTML markup usually does *not* tell us anything useful about what the text means, or what kind of data it contains. When you look at a web page in the browser, it might appear to contain a list of newspaper articles, or a table with birth rates, or a series of names with associated biographies, or whatever. But that's information that we get, as humans, from reading the page. There's (usually) no easy way to extract this information with a computer program.

HTML is also notoriously messy---web browsers are very forgiving of syntax errors and other irregularities in HTML (like mismatched or unclosed tags). (This is in contrast to data formats like JSON, where even small errors will fail to be parsed.) For this reason, we need special libraries to parse HTML into data structures that our Python programs can use, libraries that can make a "good guess" about what the structure of an HTML document is, even when that structure is written incorrectly or inconsistently.

[Beautiful Soup](http://www.crummy.com/software/BeautifulSoup/) is a Python library that parses HTML (even if it's poorly formatted) and allows us to extract and manipulate its contents. We'll be using this library in the examples that follow.

Keep in mind that there are only sketchy rules for what HTML elements "mean"---semantic information you figure out for one web page might not apply to the next. Values for `class` attributes especially are meaningful only in the context of a single page.

> Note: There's an effort to add semantic information to HTML markup called [HTML Microformats](http://microformats.org/). If sites added microformats to their markup, you'd be able to write code that could more reliably extract information from web pages, because there would be a common language for what tags with particular classes and attributes mean. Alas, microformats remain unpopular, and until the anarcho-collectivists win a greater mindshare, we can count only on our own individual readings of individual HTML documents.

### Inspecting HTML's anatomy with Developer Tools

I've crafted a very simple example of HTML for us to work with. It concerns kittens. [Here's the rendered version](http://static.decontextualize.com/kittens.html), and [here's the HTML source code](https://raw.githubusercontent.com/ledeprogram/courses/master/databases/data/kittens.html).

Now we're going to use Developer Tools in Chrome to take a look at how `kittens.html` is organized. Click on the "rendered version" link above. In Chrome, ctrl-click (or right click) anywhere on the page and select "Inspect Element." This will open Chrome's Developer Tools. Your screen should look (something) like this:

<a href="http://static.decontextualize.com/snaps/kittens-dev-tools.png"><img src="http://static.decontextualize.com/snaps/kittens-dev-tools.png" alt="kittens-dev-tools"/></a>

In the upper panel, you see the web page you're inspecting. In the lower panel, you see a version of the HTML source code, with little arrows next to some of the lines. (The little arrows allow you to collapse parts of the HTML source that are hierarchically related.) As you move your mouse over the elements in the top panel, different parts of the source code will be highlighted. Chrome is showing you which parts of the source code are causing which parts of the page to show up. Pretty spiffy!

This relationship also works in reverse: you can move your mouse over some part of the source code in the lower panel, which will highlight in the top panel what that source code corresponds to on the page. We'll be using this later to visually identify the parts of the page that are interesting to us, so we can write code that extracts the contents of those parts automatically.

###Characterizing the structure of kittens

Here's what the source code of kittens.html looks like:

	<!doctype html>
	<html>
	  <head>
	    <title>Kittens!</title>
	  </head>
	  <body>
	    <h1>Kittens and the TV Shows They Love</h1>
	    <div class="kitten">
	      <h2>Fluffy</h2>
	      <div><img src="http://placekitten.com/100/100"></div>
	      <ul class="tvshows">
	        <li><a href="http://www.imdb.com/title/tt0106145/">Deep Space Nine</a></li>
	        <li><a href="http://www.imdb.com/title/tt0088576/">Mr. Belvedere</a></li>
	      </ul>
	      Last check-up: <span class="lastcheckup">2014-01-17</span>
	    </div>
	    <div class="kitten">
	      <h2>Monsieur Whiskeurs</h2>
	      <div><img src="http://placekitten.com/150/100"></div>
	      <ul class="tvshows">
	        <li><a href="http://www.imdb.com/title/tt0106179/">The X-Files</a></li>
	        <li><a href="http://www.imdb.com/title/tt0098800/">Fresh Prince</a></li>
	      </ul>
	      Last check-up: <span class="lastcheckup">2013-11-02</span>
	    </div>
	  </body>
	</html>

This is pretty well organized HTML, but if you don't know how to read HTML, it will still look like a big jumble. Here's how I would characterize the structure of this HTML, reading in my own idea of what the meaning of the elements are.

* We have two "kittens," both of which are contained in `<div>` tags with class `kitten`.
* Each "kitten" `<div>` has an `<h2>` tag with that kitten's name.
* There's an image for each kitten, specified with an `<img>` tag.
* Each kitten has a list (a `<ul>` with class `tvshows`) of television shows, contained within `<li>` tags.
* Those list items themselves have links (`<a>` tags) with an `href` attribute that contains a link to an IMDB entry for that show.

> BONUS QUIZ: What's the parent tag of `<a href="http://www.imdb.com/title/tt0088576/">Mr. Belvedere</a>`? Both `<div class="kitten">` tags share a parent tag---what is it? What attributes are present on both `<img>` tags?

### Scraping kittens with Beautiful Soup

We've examined `kittens.html` a bit now. What we'd like to do is write some code that is going to extract information from the HTML, like "what is the last checkup date for each of these kittens?" or "what are Monsieur Whiskeur's favorite TV shows?" To do so, we need to *parse* the HTML, and create a representation of it in our program that we can manipulate with Python.

As mentioned above, HTML is hard to parse by hand. (Don't even try it. In particular, [don't parse HTML with regular expressions](http://stackoverflow.com/a/1732454).)

Beautiful Soup is a Python library that will parse the HTML for us, and give us some Python objects that we can call methods on to poke at the data contained therein. So instead of working with strings and bytes, we can work with 

Beautiful Soup is available on PyPI, which means you can install it with `pip`:

    pip3 install beautifulsoup4

Beautiful Soup only parses HTML. It's left to us to actually *get* the HTML from somewhere. Because we're doing web scraping, most likely we'll want to download the HTML directly from the web site that hosts it. For that, we'll use the `urlopen` function in the built-in Python library `urllib.request`:

In [9]:
from urllib.request import urlopen

html_str = urlopen("https://raw.githubusercontent.com/ledeprogram/courses/master/databases/data/kittens.html").read()

Now `html_str` is a byte string that contains the HTML source code of the page in question:

In [10]:
print(html_str)

b'<!doctype html>\n<html>\n\t<head>\n\t\t<title>Kittens!</title>\n\t</head>\n\t<body>\n\t\t<h1>Kittens and the TV Shows They Love</h1>\n\t\t<div class="kitten">\n\t\t\t<h2>Fluffy</h2>\n\t\t\t<div><img src="http://placekitten.com/100/100"></div>\n\t\t\t<ul class="tvshows">\n\t\t\t\t<li>\n\t\t\t\t\t<a href="http://www.imdb.com/title/tt0106145/">Deep Space Nine</a>\n\t\t\t\t</li>\n\t\t\t\t<li>\n\t\t\t\t\t<a href="http://www.imdb.com/title/tt0088576/">Mr. Belvedere</a>\n\t\t\t\t</li>\n\t\t\t</ul>\n\t\t\tLast check-up: <span class="lastcheckup">2014-01-17</span>\n\t\t</div>\n\t\t<div class="kitten">\n\t\t\t<h2>Monsieur Whiskeurs</h2>\n\t\t\t<div><img src="http://placekitten.com/150/100"></div>\n\t\t\t<ul class="tvshows">\n\t\t\t\t<li>\n\t\t\t\t\t<a href="http://www.imdb.com/title/tt0106179/">The X-Files</a>\n\t\t\t\t</li>\n\t\t\t\t<li>\n\t\t\t\t\t<a href="http://www.imdb.com/title/tt0098800/">Fresh Prince</a>\n\t\t\t\t</li>\n\t\t\t</ul>\n\t\t\tLast check-up: <span class="lastcheckup">2013-1

That looks like a mess but it's apparent that we've obtained the data as desired.

> The `b` in front of the string above indicates that this is a special kind of string in Python called a "byte string," which is essentially "raw" data that hasn't yet been converted into Unicode. [See this document for more information on byte strings and Unicode in Python 3](https://docs.python.org/3/howto/unicode.html).

Now we need to create a Beautiful Soup object from that data. Here's how to go about doing that:

In [21]:
from bs4 import BeautifulSoup

document = BeautifulSoup(html_str, "html.parser")
print(type(document))

<class 'bs4.BeautifulSoup'>


The `BeautifulSoup` function creates a new Beautiful Soup object. It takes two parameters: the string containing the HTML data, and a string that designates which underlying parser to use to build the parsed version of the document. (Leave this as `"html.parser"`.) I've assigned this object to the variable `document`. This object supports a number of interesting methods that allow us to dig into the contents of the HTML. Primarily what we'll be working with are `Tag` objects and `ResultSet` objects, which are essentially just lists of `Tag` objects.

### Finding a tag

HTML documents are composed of tags. To represent this, Beautiful Soup has a type of value that represents tags. We can use the `.find()` method of the `BeautifulSoup` object to find a tag that matches a particular tag name. For example:

In [22]:
h1_tag = document.find('h1')
print(type(h1_tag))

<class 'bs4.element.Tag'>


A `Tag` object has several interesting attributes and methods. The `string` attribute of a `Tag` object, for example, returns a string representing that tag's contents:

In [23]:
print(h1_tag.string)

Kittens and the TV Shows They Love


You can access the attributes of a tag by treating the tag object as though it were a dictionary, using the square-bracket index syntax, with the name of the attribute whose value you want as a string inside the brackets. For example, to print out the `src` attribute of the first `<img>` tag in the document:

In [24]:
img_tag = document.find('img')
print(img_tag['src'])

http://placekitten.com/100/100


> Note: You might have noticed that there is more than one `<img>` tag in `kittens.html`! If more than one tag matches the name you pass to `.find()`, it returns only the *first* matching tag. (A better name for `.find()` might be `find_first`.)

### Finding multiple tags

It's often the case that we want to find not just one tag that matches particular criteria, but ALL tags matching those criteria. For that, we use the `.find_all()` method of the `BeautifulSoup` object. For example, to find all `h2` tags in the document:

In [25]:
h2_tags = document.find_all('h2')
print(type(h2_tags))
[tag.string for tag in h2_tags]

<class 'bs4.element.ResultSet'>


['Fluffy', 'Monsieur Whiskeurs']

Both the `.find()` and `.find_all()` methods can search not just for tags with particular names, but also for tags that have particular attributes. For that, we use the `attrs` keyword argument, giving it a dictionary that associates attribute names as keys and the desired attribute value as values. For example, to find all `span` tags with a `class` attribute of `lastcheckup`:

In [26]:
checkup_tags = document.find_all('span', attrs={'class': 'lastcheckup'})
[tag.string for tag in checkup_tags]

['2014-01-17', '2013-11-02']

> Note: Beautiful Soup's `.find()` and `.find_all()` methods are actually more powerful than we're letting on here. [Check out the details in the official Beautiful Soup documentation.](http://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-all)

### Finding tags within tags

Let's say that we wanted to print out a list of the name of each kitten, along with a list of the names of that kitten's favorite TV shows. In other words, we want to print out something that looks like this:

    Fluffy: Deep Space Nine, Mr. Belvedere
    Monsieur Whiskeurs: The X-Files, Fresh Prince
    
In order to do this, we need to find *not just* tags with particular names, but tags with *particular hierarchical relationships* with other tags. I.e., we need to identify all of the kittens, and then find the shows that belong to that kitten. This kind of search is made easy by the fact that you can use `.find()` and `.find_all()` methods not just on the entire document, but on *individual tags*. When you use these methods on tags, they search for matching tags that are specifically *children of* the tag that you call them on.

In our kittens example, we can see that information about individual kittens is grouped together under `<div>` tags with a `class` attribute of `kitten`. So, to find a list of all `<div>` tags with `class` set to `kitten`, we might do this:

In [28]:
kitten_tags = document.find_all("div", attrs={"class": "kitten"})

Now, we'll loop over that list of tags and find, inside each of them, the `<h2>` tag that is its child:


In [29]:
for kitten_tag in kitten_tags:
    h2_tag = kitten_tag.find('h2')
    print(h2_tag.string)

Fluffy
Monsieur Whiskeurs


Now, we'll go one extra step. Looping over all of the kitten tags, we'll find not just the `<h2>` tag with the kitten's name, but all `<a>` tags (which contain the names of the TV shows that we were looking for):

In [30]:
for kitten_tag in kitten_tags:
    h2_tag = kitten_tag.find('h2')
    a_tags = kitten_tag.find_all('a')
    a_tag_strings = [tag.string for tag in a_tags]
    a_tag_strings_joined = ", ".join(a_tag_strings)
    print(h2_tag.string + ": " + a_tag_strings_joined)

Fluffy: Deep Space Nine, Mr. Belvedere
Monsieur Whiskeurs: The X-Files, Fresh Prince


> EXERCISE: Modify the code above to print out a list of kitten names along with the last check-up date for that kitten.

> EXTRA FUN EXERCISE: Rewrite the code above to create a dictionary that maps kitten names to a list of links to that kitten's favorite shows. I.e., you should end up with a dictionary that looks like this:

    {u'Fluffy': [u'http://www.imdb.com/title/tt0106145/',
      u'http://www.imdb.com/title/tt0088576/'],
     u'Monsieur Whiskeurs': [u'http://www.imdb.com/title/tt0106179/',
      u'http://www.imdb.com/title/tt0098800/']}

### Finding sibling tags

Often, the tags we're looking for don't have a distinguishing characteristic, like a `class` attribute, that allows us to find them using `.find()` and `.find_all()`, and the tags also aren't in a parent-child relationship. This can be tricky! Take the following HTML snippet, for example:    

In [31]:
cheese_html = """
<h2>Camembert</h2>
<p>A soft cheese made in the Camembert region of France.</p>

<h2>Cheddar</h2>
<p>A yellow cheese made in the Cheddar region of... France, probably, idk whatevs.</p>
"""

If our task was to create a dictionary that maps the name of the cheese to the description that follows in the `<p>` tag directly afterward, we'd be out of luck. Fortunately, Beautiful Soup has a `.find_next_sibling()` method, which allows us to search for the next tag that is a *sibling* of the tag you're calling it on (i.e., the two tags share a parent), that also matches particular criteria. So, for example, to accomplish the task outlined above:

In [33]:
document = BeautifulSoup(cheese_html, "html.parser")
cheese_dict = {}
for h2_tag in document.find_all('h2'):
    cheese_name = h2_tag.string
    cheese_desc_tag = h2_tag.find_next_sibling('p')
    cheese_dict[cheese_name] = cheese_desc_tag.string

cheese_dict

{'Camembert': 'A soft cheese made in the Camembert region of France.',
 'Cheddar': 'A yellow cheese made in the Cheddar region of... France, probably, idk whatevs.'}

You now know most of what you need to know to scrape web pages effectively. Good job!

### When things go wrong with Beautiful Soup

A number of things might go wrong with Beautiful Soup. You might, for example, search for a tag that doesn't exist in the document:

In [34]:
footer_tag = document.find("footer")

Beautiful Soup doesn't return an error if it can't find the tag you want. Instead, it returns `None`:

In [36]:
print(footer_tag)

None


If you try to call a method on the object that Beautiful Soup returned anyway, you might end up with an error like this:

In [37]:
footer_tag.find("p")

AttributeError: 'NoneType' object has no attribute 'find'

You might also inadvertently try to get an attribute of a tag that wasn't actually found. You'll get a similar error in that case:

In [38]:
footer_tag['title']

TypeError: 'NoneType' object is not subscriptable

Whenever you see something like `TypeError: 'NoneType' object is not subscriptable`, it's a good idea to check to see whether your method calls are indeed finding the thing you were looking for.

However, the `.find_all()` method will return an empty list if it doesn't find any of the tags you wanted:

In [42]:
footer_tags = document.find_all("footer")
print(footer_tags)

[]


If you attempt to access one of the elements of this regardless...

In [43]:
print(footer_tags[0].string)

IndexError: list index out of range

...you'll get an `IndexError`.

## A real-world example, sort of

The rocky blue golf ball we live on is simply filthy with HTML pages ripe for scraping. As an example today, we're going to take a look at [this one](http://www.journalism.columbia.edu/page/10/10?category_ids%5B%5D=2&category_ids%5B%5D=3&category_ids%5B%5D=37). That's right, the faculty list page for the Columbia Graduate School of Journalism. Whoa.

Our task is to create a list of every faculty member, along with their title, and a link to their photo (if any). Let's say that, in the end, we want to end up with a list of dictionaries that looks like this:

    [
        {'name': 'Adkison, Abbey', 'title': 'Digital Media Coordinator', 'img_src': None,
        {'name': 'Barclay, Dolores', 'title': 'Adjunct Faculty', 'img_src': 'http://www.journalism.columbia.edu/system/photos/1943/default/Dolores-Barclay.gif?1365711292',
        {'name': 'Baum, Geraldine', 'title': 'Adjunct Faculty', 'img_src': None}
        ...
    ]

(We could then do fun things with this data, like insert it into a SQL database or something.)

We're going to make some mistakes along the way, so you know what it looks like when mistakes happen and so you'll learn a little bit about how to recover from them.

The first step in this task? Use Developer Tools to find the elements we're looking for. In this screenshot, I'm mousing over the unit on the page that seems to contain the information I'm looking for. I've "expanded" some of the collapsed code sections to make it easier to see the hierarchy of tags.

<a href="http://static.decontextualize.com/snaps/Google%20Chrome.png"><img src="http://static.decontextualize.com/snaps/Google%20Chrome.png" alt="Google Chrome"/></a>

Based on what I'm seeing here, I can start to formulate a plan to scrape the document. Here's what I came up with:

* It looks like each faculty member has an `<li>` tag, so I'll find all of those.
* For each `<li>` tag, I need to find an `<img>` tag---specifically, I need to grab the `src` attribute from that tag.
* The faculty member's name is inside an `<a>` tag---specifically, an `<a>` tag inside of an `<h4>` tag.
* The faculty member's title seems to be located inside a `<p>` tag with `class` attribute `description`.

Let's write some code to do that!

In [45]:
html_str = urlopen("http://www.journalism.columbia.edu/page/10/10?category_ids%5B%5D=2&category_ids%5B%5D=3&category_ids%5B%5D=37").read()
document = BeautifulSoup(html_str, "html.parser")
faculty_list = []
for faculty_tag in document.find_all('li'):
    # create empty dictionary to store this faculty member
    faculty_dict = {}
    # faculty name
    h4_tag = faculty_tag.find('h4')
    a_tag = h4_tag.find('a')
    faculty_dict['name'] = a_tag.string
    # image URL
    img_tag = faculty_tag.find('img')
    faculty_dict['img_src'] = img_tag['src']
    # title
    p_tag = faculty_tag.find('p', attrs={'class': 'description'})
    faculty_dict['title'] = p_tag.string
    # append to list
    faculty_list.append(faculty_dict)
faculty_list

AttributeError: 'NoneType' object has no attribute 'find'

Looks good, but wait! When we run it, we get an error: `AttributeError: 'NoneType' object has no attribute 'find'`. Specifically, the `h4_tag`, whose `.find()` method we're trying to use, seems to be `None`... which means that there wasn't an `h4` tag where we expected there to be one. To diagnose the problem, let's simplify our code a little bit to see where the problem is. (I used the range syntax `[:5]` in the `for` loop here, so that we'll grab just the first five results---that should be enough to diagnose our problem.)

In [47]:
html_str = urlopen("http://www.journalism.columbia.edu/page/10/10?category_ids%5B%5D=2&category_ids%5B%5D=3&category_ids%5B%5D=37").read()
document = BeautifulSoup(html_str, "html.parser")
for faculty_tag in document.find_all('li')[:5]:
    print(faculty_tag)

<li><a href="/page/1-about-the-school/1">About the School</a> </li>
<li><a href="/page/7-our-programs/7">Academic Programs</a> </li>
<li><a href="/page/13-career-development/13">Career Development</a> </li>
<li><a href="/page/14-events-calendar/14">Events</a> </li>
<li><a class="active" href="/page/10-full-time-adjunct-visiting-faculty/10">Faculty</a></li>


When you `print` a `Tag` object, BeautifulSoup displays the source code for those tags. Here we can see that the `<li>` tags we've found aren't quite the `<li>` tags we were looking for---these seem to be `<li>` tags from another part of the page! Whoops. I guess we need to be more specific about which `<li>` tags we want. How do we do that, though? Let's go back to Developer Tools.

<a href="http://static.decontextualize.com/snaps/Google%20Chrome.png"><img src="http://static.decontextualize.com/snaps/Google%20Chrome.png" alt="Google Chrome"/></a>

Now it *looks* like all of the relevant `<li>` tags have a single parent tag---`<ul class="experts-list">`. So what we need to do is find not *all `<li>` tags on the page*, but *only those `<li>` tags that are children of this particular `<ul>` tag*. Here's some revised code to do just that:

In [48]:
experts_ul_tag = document.find('ul', attrs={'class': 'experts-list'})
for faculty_tag in experts_ul_tag.find_all('li')[:5]:
    print(faculty_tag)

<li class="label" id="goto-a"></li>
<li>
<div class="content">
<h4><a href="/profile/228-abbey-adkison/10">Adkison, Abbey </a></h4>
<p class="description">Assistant Director, Multi-Media Journalism</p>
</div>
</li>
<li>
<a href="/profile/536-daniel-alarcon/10"><img alt="Daniel_a" border="1" height="90" src="/system/photos/3771/default/daniel_a.jpg?1408652577" width="110"/></a>
<div class="content">
<h4><a href="/profile/536-daniel-alarcon/10">Alarcón, Daniel</a></h4>
<p class="description">Assistant Professor of Broadcast Journalism</p>
<p class="expertise">Expertise: BROADCAST</p>
</div>
</li>
<li class="label" id="goto-b"></li>
<li>
<a href="/profile/387-dolores-barclay/10"><img alt="Dolores-barclay" border="1" height="90" src="/system/photos/1943/default/Dolores-Barclay.gif?1365711292" width="110"/></a>
<div class="content">
<h4><a href="/profile/387-dolores-barclay/10">Barclay, Dolores </a></h4>
<p class="description">Adjunct Faculty</p>
</div>
</li>


This looks a little bit better, but we've still got a few weird things: namely, there are some `<li>` tags, (those with a `class` attribute of `label`) which don't seem to contain `h4` tags---or any other content we're interested in at all. We need to put in a check so our code will disregard any `<li>` tags like this. Here's another attempt:

In [49]:
experts_ul_tag = document.find('ul', attrs={'class': 'experts-list'})
for faculty_tag in experts_ul_tag.find_all('li')[:5]:
    h4_tag = faculty_tag.find('h4')
    if h4_tag is None:
        continue
    print(h4_tag)

<h4><a href="/profile/228-abbey-adkison/10">Adkison, Abbey </a></h4>
<h4><a href="/profile/536-daniel-alarcon/10">Alarcón, Daniel</a></h4>
<h4><a href="/profile/387-dolores-barclay/10">Barclay, Dolores </a></h4>


Okay, now we're in business. At last. Let's put this code together with the previous example.

In [50]:
html_str = urlopen("http://www.journalism.columbia.edu/page/10/10?category_ids%5B%5D=2&category_ids%5B%5D=3&category_ids%5B%5D=37").read()
document = BeautifulSoup(html_str, "html.parser")
faculty_list = []
experts_ul_tag = document.find('ul', attrs={'class': 'experts-list'})
for faculty_tag in experts_ul_tag.find_all('li'):
    # create empty dictionary to store this faculty member
    faculty_dict = {}
    # faculty name
    h4_tag = faculty_tag.find('h4')
    if h4_tag is None:
        continue
    a_tag = h4_tag.find('a')
    faculty_dict['name'] = a_tag.string
    # image URL
    img_tag = faculty_tag.find('img')
    faculty_dict['img_src'] = img_tag['src']
    # title
    p_tag = faculty_tag.find('p', attrs={'class': 'description'})
    faculty_dict['title'] = p_tag.string
    # append to list
    faculty_list.append(faculty_dict)
faculty_list

TypeError: 'NoneType' object is not subscriptable

ARGH! Another `'NoneType' object is not subscriptable`! It looks like the problem this time is with the `img_tag` variable. In particular, if we examine the source code, we find that faculty members without head shots don't even have an `<img>` tag in their `<div>`s. So let's add one more thing to fix that---we'll check to see if the `<img>` tag is present, and only then will we attempt to get its `src` attribute:

In [51]:
html_str = urlopen("http://www.journalism.columbia.edu/page/10/10?category_ids%5B%5D=2&category_ids%5B%5D=3&category_ids%5B%5D=37").read()
document = BeautifulSoup(html_str, "html.parser")
faculty_list = []
experts_ul_tag = document.find('ul', attrs={'class': 'experts-list'})
for faculty_tag in experts_ul_tag.find_all('li'):
    # create empty dictionary to store this faculty member
    faculty_dict = {}
    # faculty name
    h4_tag = faculty_tag.find('h4')
    if h4_tag is None:
        continue
    a_tag = h4_tag.find('a')
    faculty_dict['name'] = a_tag.string
    # image URL: if <img> tag found, grab its src. if not, use None
    img_tag = faculty_tag.find('img')
    if img_tag is None:
        faculty_dict['img_src'] = None
    else:
        faculty_dict['img_src'] = img_tag['src']
    # title
    p_tag = faculty_tag.find('p', attrs={'class': 'description'})
    faculty_dict['title'] = p_tag.string
    # append to list
    faculty_list.append(faculty_dict)
faculty_list

[{'img_src': None,
  'name': 'Adkison, Abbey ',
  'title': 'Assistant Director, Multi-Media Journalism'},
 {'img_src': '/system/photos/3771/default/daniel_a.jpg?1408652577',
  'name': 'Alarcón, Daniel',
  'title': 'Assistant Professor of Broadcast Journalism'},
 {'img_src': '/system/photos/1943/default/Dolores-Barclay.gif?1365711292',
  'name': 'Barclay, Dolores ',
  'title': 'Adjunct Faculty'},
 {'img_src': None, 'name': 'Baum, Geraldine', 'title': 'Adjunct Faculty'},
 {'img_src': '/system/photos/2056/default/EBell_112811.jpg?1322508884',
  'name': 'Bell, Emily',
  'title': 'Professor of Professional Practice & Director, Tow Center for Digital Journalism'},
 {'img_src': '/system/photos/2057/default/HBenedict_112811.jpg?1322509591',
  'name': 'Benedict, Helen ',
  'title': 'Professor'},
 {'img_src': '/system/photos/2982/default/Bennet_John.gif?1365697019',
  'name': 'Bennet, John ',
  'title': 'Adjunct Faculty'},
 {'img_src': '/system/photos/2984/default/Bennett_Rob.gif?1365706134',
  

It worked! So good. Now we can do fun stuff with the data, like using a list comprehension to count how many are adjunct faculty:

In [55]:
adjuncts = [prof['name'] for prof in faculty_list if prof['title'] == 'Adjunct Faculty']
print(len(adjuncts))

88


## Further reading

* [Chapter 11](https://automatetheboringstuff.com/chapter11/) from Al Sweigart's [Automate the Boring Stuff with Python](https://automatetheboringstuff.com/) is another good take on this material (and discusses a wider range of techniques).
* [The official Beautiful Soup documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) provides a systematic walkthrough of the library's functionality. If you find yourself thinking, "it really should be easy to do the thing that I want to do, why isn't it easier?" then check the documentation! Leonard's probably already thought of a way to make it easier and implemented a feature in the code to help you out.
* Beautiful Soup is the best scraping library out there for quick jobs, but if you have a larger site that you need to scrape, you might look into [Scrapy](http://scrapy.org/), which bundles a good parser with a framework for writing web "spiders" (i.e., programs that parse web pages and follow the links found there, in order to make a catalog of an entire web site, not just a single web page).